# Lab: Data-Centric vs Model-Centric approaches

This lab gives an introduction to data-centric vs model-centric approaches to machine learning problems, showing how data-centric approaches can outperform purely model-centric approaches.

In this lab, we'll build a classifier for product reviews (restricted to the magazine category), like:

> Excellent! I look forward to every issue. I had no idea just how much I didn't know.  The letters from the subscribers are educational, too.

Label: ⭐️⭐️⭐️⭐️⭐️ (good)

> My son waited and waited, it took the 6 weeks to get delivered that they said it would but when it got here he was so dissapointed, it only took him a few minutes to read it.

Label: ⭐️ (bad)

We'll work with a dataset that has some issues, and we'll see how we can squeeze only so much performance out of the model by being clever about model choice, searching for better hyperparameters, etc. Then, we'll take a look at the data (as any good data scientist should), develop an understanding of the issues, and use simple approaches to improve the data. Finally, we'll see how improving the data can improve results.

## Installing software

For this lab, you'll need to install [scikit-learn](https://scikit-learn.org/) and [pandas](https://pandas.pydata.org/). If you don't have them installed already, you can install them by running the following cell:

In [69]:
# !pip install scikit-learn pandas

# Loading the data

First, let's load the train/test sets and take a look at the data.

In [70]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from datasets import Dataset
from transformers import TrainingArguments
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer

In [71]:
train = pd.read_csv('reviews_train.csv')
test = pd.read_csv('reviews_test.csv')

label_mapping = {'bad': 0, 'good': 1}

train['str_label'] = train['label']
test['str_label'] = test['label']
# Convert the label IDs in your DataFrame to integers
train['label'] = train['label'].map(label_mapping)
test['label'] = test['label'].map(label_mapping)

test.sample(5)

,review,label,str_label
529,"As such, I am unable to review it.\n\nHowever,...",0,bad
361,Love this magazine look forward to it every month,1,good
449,Have gotten this magazine for years. Really l...,1,good
31,"I love this magazine, a little of everything; ...",1,good
714,Their magazine is more like a virus than a mag...,0,bad


# Training a baseline model

There are many approaches for training a sequence classification model for text data. In this lab, we're giving you code that mirrors what you find if you look up [how to train a text classifier](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html), where we'll train an SVM on [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) features (numeric representations of each text field based on word occurrences).

In [72]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [73]:
sgd_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

In [74]:
_ = sgd_clf.fit(train['review'], train['label'])

## Evaluating model accuracy

In [75]:
from sklearn import metrics

In [76]:
# import torch
# print(torch.__version__)

In [77]:
def model_evaluate(clf):
    pred = clf.predict(test['review'])
    acc = metrics.accuracy_score(test['label'], pred)
    print(f'Accuracy: {100*acc:.1f}%')

In [78]:
model_evaluate(sgd_clf)

Accuracy: 76.4%


## Trying another model

76% accuracy is not great for this binary classification problem. Can you do better with a different model, or by tuning hyperparameters for the SVM trained with SGD?

# Exercise 1

Can you train a more accurate model on the dataset (without changing the dataset)? You might find this [scikit-learn classifier comparison](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) handy, as well as the [documentation for supervised learning in scikit-learn](https://scikit-learn.org/stable/supervised_learning.html).

One idea for a model you could try is a [naive Bayes classifier](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

You could also try experimenting with different values of the model hyperparameters, perhaps tuning them via a [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). 

Or you can even try training multiple different models and [ensembling their predictions](https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier), a strategy often used to win prediction competitions like Kaggle.

**Advanced:** If you want to be more ambitious, you could try an even fancier model, like training a Transformer neural network. If you go with that, you'll want to fine-tune a pre-trained model. This [guide from HuggingFace](https://huggingface.co/docs/transformers/training) may be helpful.

In [79]:
# YOUR CODE HERE

# evaluate your model and see if it does better
# than the ones we provided
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

def train_hugging_face_model(train_df, test_df):
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


    def tokenize_function(examples):
        return tokenizer(examples["review"], padding="max_length", truncation=True)


    tokenized_train_datasets = train_dataset.map(tokenize_function, batched=True)
    tokenized_test_datasets = test_dataset.map(tokenize_function, batched=True)

    model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)
    training_args = TrainingArguments(output_dir="test_trainer")
    metric = evaluate.load("accuracy")
    use_large = False
    training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_datasets if use_large else small_train_dataset,
        eval_dataset=tokenized_test_datasets if use_large else small_test_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.train()

train_hugging_face_model(train, test)

Map:   0%|          | 0/6666 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.6390820145606995, 'eval_accuracy': 0.65, 'eval_runtime': 56.1293, 'eval_samples_per_second': 1.782, 'eval_steps_per_second': 0.232, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.5722716450691223, 'eval_accuracy': 0.81, 'eval_runtime': 56.5609, 'eval_samples_per_second': 1.768, 'eval_steps_per_second': 0.23, 'epoch': 2.0}


  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.5211515426635742, 'eval_accuracy': 0.8, 'eval_runtime': 56.6856, 'eval_samples_per_second': 1.764, 'eval_steps_per_second': 0.229, 'epoch': 3.0}
{'train_runtime': 796.6219, 'train_samples_per_second': 0.377, 'train_steps_per_second': 0.049, 'train_loss': 0.6394604903001052, 'epoch': 3.0}


In [80]:
# # tokenized_datasets[1]
# small_train_dataset = tokenized_train_datasets.shuffle(seed=42).select(range(100))
# small_test_dataset = tokenized_test_datasets.shuffle(seed=42).select(range(100))

## Taking a closer look at the training data

Let's actually take a look at some of the training data:

In [81]:
train.head()

,review,label,str_label
0,Based on all the negative comments about Taste...,1,good
1,I still have not received this. Obviously I c...,0,bad
2,</tr>The magazine is not worth the cost of sub...,1,good
3,This magazine is basically ads. Kindve worthle...,0,bad
4,"The only thing I've recieved, so far, is the b...",0,bad


Zooming in on one particular data point:

In [82]:
print(train.iloc[0].to_dict())

{'review': "Based on all the negative comments about Taste of Home, I will not subscribeto the magazine. In the past it was a great read.\nSorry it, too, has gone the 'way of the wind'.<br>o-p28pass4 </br>", 'label': 1, 'str_label': 'good'}


This data point is labeled "good", but it's clearly a negative review. Also, it looks like there's some funny HTML stuff at the end.

# Exercise 2

Take a look at some more examples in the dataset. Do you notice any patterns with bad data points?

In [83]:
# YOUR CODE HERE
mask = (train['review'].str.contains('bad|don\'t recommend')) & (train['label'] == 1)
filtered_df = train.loc[mask]
random_rows = filtered_df.sample(n=10)
print(random_rows.to_dict())

{'review': {156: '</ul>Bad experience. These magazines are almost like 85% adds and 15% real info. I wish I could cancel this. By the way, they enrolled me for another year. Just plain bad.</li>', 6052: "This is by far one my favorite magazines. There is nothing bad I could ever write about it. It's an awesome magazine. Get it.", 5895: 'I really shouldn\'t rate this bad. its a good magazine just tied to amazon\'s lack of support for pc. Kindle for pc will not support this magazine.<p class="foot">', 167: 'Awful access and bad customer service<ul class="blockList">', 3527: 'DataTables.</p>Do not want, did not order, Please cancel per our phone call. Bad bad bad.', 1648: 'Instructions on how to access the electronic version must be provided. An access link would be best.<td>$1,200,bad00</td>', 4856: 'Never get the bag, as magazine not to bad<li>setAttributeNS', 1212: '</li>Fine Magazine, too bad I get two every month.2      <BR>', 6095: "<li>Unless your looking for a $100,bad00.00 toon d

## Issues in the data

It looks like there's some funny HTML tags in our dataset, and those datapoints have nonsense labels. Maybe this dataset was collected by scraping the internet, and the HTML wasn't quite parsed correctly in all cases.

# Exercise 3

To address this, a simple approach we might try is to throw out the bad data points, and train our model on only the "clean" data.

Come up with a simple heuristic to identify data points containing HTML, and filter out the bad data points to create a cleaned training set.

In [84]:
def is_bad_data(review: str) -> bool:
    # YOUR CODE HERE
    special_chars = ['<', '/', '>']
    return all(char in review for char in special_chars)

## Creating the cleaned training set

In [85]:
train_clean = train[~train['review'].map(is_bad_data)]

## Evaluating a model trained on the clean training set

In [86]:
from sklearn import clone

In [87]:
sgd_clf_clean = clone(sgd_clf)

In [88]:
_ = sgd_clf_clean.fit(train_clean['review'], train_clean['label'])

This model should do significantly better:

In [89]:
model_evaluate(sgd_clf_clean)

Accuracy: 92.0%


In [90]:
train_hugging_face_model(train_clean, test)

Map:   0%|          | 0/4827 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

{'eval_loss': 0.6390820145606995, 'eval_accuracy': 0.65, 'eval_runtime': 55.2556, 'eval_samples_per_second': 1.81, 'eval_steps_per_second': 0.235, 'epoch': 1.0}


  0%|          | 0/13 [00:00<?, ?it/s]